In [1]:
import pandas as pd
import numpy as np
import sys

### Import Oracle Creator

In [2]:
sys.path.append("../../../../Tracer/src")
from DataHandling.TraceCreator import create_trace_matrix_from_trace_list

def create_trace_list(path_to_oracle):
    oracle = open(path_to_oracle).read().split("\n")
    trace_list = []
    for trace_item in oracle:
        if len(trace_item) > 0:
            source, targets = trace_item.split(":")
            for target in targets.split(" "):
                if len(target) != 0:
                    trace_list.append(" ".join([source, target]))
                else:
                    trace_list.append(" ".join([source, ""]))
    return create_trace_matrix_from_trace_list(trace_list)

# Original Oracles

In [3]:
r2d = pd.read_csv("../../Oracles/DataFrames/Level_1_to_Level_2.csv").set_index("id")
d2c = pd.read_csv("../../Oracles/DataFrames/Level_2_to_Level_3.csv").set_index("id")

### Import New Oracle

In [4]:
c2c = create_trace_list("../Originals/TracedMatrices/CC_CC.txt")
c2c.head(3)

,132.txt,141.txt,146.txt,137.txt,147.txt,145.txt,152.txt,124.txt,151.txt,129.txt,...,119.txt,120.txt,159.txt,154.txt,127.txt,134.txt,150.txt,128.txt,116.txt,123.txt
id,,,,,,,,,,,,,,,,,,,,,
132.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### R 2 D

In [5]:
def get_similar_classes(c_id):
    similar_classes = []
    if c_id in c2c.index:
        for c2_id in c2c.columns:
            if c2c.loc[c_id][c2_id] == 1:
                similar_classes.append(c2_id)
    return similar_classes

In [6]:
missing_traces = []
n_missing_traces = 0
n_confirmed_traces = 0
n_undefined_classes = 0
table = d2c
for d_id in table.index:
    for c_id in table.columns:
        if table.loc[d_id][c_id] == 1:
            similar_classes = get_similar_classes(c_id)
            for s_class in similar_classes:
                if s_class not in table.columns:
                    n_undefined_classes = n_undefined_classes + 1
                elif s_class in table.columns and table.loc[d_id][s_class] == 0:
                    n_missing_traces = n_missing_traces + 1
                    missing_traces.append((d_id, s_class))
                elif s_class in table.columns and table.loc[d_id][s_class] == 1:
                    n_confirmed_traces = n_confirmed_traces + 1
                    
print("Number of Missing Traces: ", len(missing_traces))
print("NUmber of confirmed traces: ", n_confirmed_traces)
print("Number of undefined classes: ", n_undefined_classes)

Number of Missing Traces:  71
NUmber of confirmed traces:  61
Number of undefined classes:  49


# Add missing traces

In [7]:
for mt in missing_traces:
    d2c.loc[mt[0]][mt[1]] = 1

In [8]:
d2c.to_csv("../../Oracles/DataFrames/Level_2_to_Level_3.csv")
print("Done!")

Done!
